###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests

# Veteran Status by Age for the GNRC Senior Centers

This data is not available on MySidewalk, where these senior center profiles are hosted. This notebook contains the API pulls and the data cleaning process necessary to include this as outside data in these profiles.

This data will be collected at the county level for the following counties in the State of TN:  
+ Montgomery (Ajax Turner Senior Citizens Center)  
+ Cheatham (Ashland City Senior Citizens Center)  
+ Robertson (Byrum-Porter Senior Center)  
+ Dickson (Dickson Senior Citizens Center)  
+ Williamson (FiftyForward College Grove Senior Citizens Center)  
+ Davidson (FiftyForward Donelson Senior Citizens Center)  
+ Davidson (FiftyForward Knowles Senior Citizens Center)  
+ Sumner (Gallatin Senior Center)  
+ Sumner (Senior Citizens of Hendersonville)  
+ Houston (J.D. Lewis Senior Center)  
+ Rutherford (LaVerge Senior Center)  
+ Wilson (Mt. Juliet Senior Center)  
+ Robertson (Robertson County Senior Center)  
+ Rutherford (St. Clair Street Senior Center)  
+ Stewart (Stewart County Senior Citizens Center)  
+ Trousdale (Trousdale County/Hartsville Senior Center)  
+ Humphreys (Torrey Johnson Senior Center)  

In [22]:
GNRC = ['161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149'] #Rutherford

In [23]:
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'

In [24]:
#veteran status 60+ over 18 civilian population
data_appended = []
for i in GNRC:
    url_str= 'https://api.census.gov/data/2021/acs/acs5/subject?key='+api_key
    predicates= {}
    get_vars= ["NAME", 'S0102_C02_041E'] #the only variable is total population
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:{}".format(i)
    predicates["in"]= "state:47"
    data= requests.get(url_str, params= predicates)
    col_names = ['County', 'vet60+', 'StateFIPS', 'CountyFIPS']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:])
    data['Region'] = 'GNRC'
    data_appended.append(data)
    
data_appended = pd.concat(data_appended)
data = data_appended.reset_index(drop = True)
print('Your API call is complete.')

Your API call is complete.


In [25]:
data.head(13)

,County,vet60+,StateFIPS,CountyFIPS,Region
0,"Stewart County, Tennessee",None,47,161,GNRC
1,"Montgomery County, Tennessee",30520,47,125,GNRC
2,"Houston County, Tennessee",None,47,083,GNRC
3,"Humphreys County, Tennessee",None,47,085,GNRC
4,"Dickson County, Tennessee",12744,47,043,GNRC
5,"Cheatham County, Tennessee",9054,47,021,GNRC
6,"Robertson County, Tennessee",15279,47,147,GNRC
7,"Sumner County, Tennessee",42986,47,165,GNRC
8,"Davidson County, Tennessee",125124,47,037,GNRC
9,"Wilson County, Tennessee",31716,47,189,GNRC


We can see that the subject tables aren't available for Stewart, Houston, Humphreys, or Trousdale Counties. We'll have to do the more granular age breakdowns from the ACS tables. I'd like to get # 65+, # all, percent of veterans who are seniors, and percent of senior population who are veterans. For this I'll need total civilian veteran population, total senior population from this data series, and total # 65+ by whatever categories this series has it.

In [26]:
data_appended = []
for i in GNRC:
    url_str= 'https://api.census.gov/data/2021/acs/acs5?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID",'B21001_016E','B21001_017E','B21001_019E','B21001_020E','B21001_034E','B21001_035E','B21001_037E','B21001_038E','B21001_002E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:{}".format(i)
    predicates["in"]= "state:47"
    data= requests.get(url_str, params= predicates)
    col_names = ['County', 'GEOID','m65_74','mv65_74','m75+','mv75+','f65_74','fv65_74','f75+','fv75+','totalv','StateFIPS','CountyFIPS']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype = str)
    data['Region'] = 'GNRC'
    data_appended.append(data)
    
data_appended = pd.concat(data_appended)
data = data_appended.reset_index(drop = True)
print('Your API call is complete')

Your API call is complete


In [27]:
data.head()

,County,GEOID,m65_74,mv65_74,m75+,mv75+,f65_74,fv65_74,f75+,fv75+,totalv,StateFIPS,CountyFIPS,Region
0,"Stewart County, Tennessee",0500000US47161,803,353,409,185,868,16,608,14,1509,47,161,GNRC
1,"Montgomery County, Tennessee",0500000US47125,6007,3300,2995,1916,6923,425,4200,55,28547,47,125,GNRC
2,"Houston County, Tennessee",0500000US47083,485,172,280,191,499,0,406,7,813,47,083,GNRC
3,"Humphreys County, Tennessee",0500000US47085,1095,279,627,348,1112,0,944,3,1342,47,085,GNRC
4,"Dickson County, Tennessee",0500000US47043,2556,570,1335,683,2873,45,1890,0,3141,47,043,GNRC


In [28]:
#make columns integers that we need for mathematical operations
cols = ['m65_74','mv65_74','m75+','mv75+','f65_74','fv65_74','f75+','fv75+','totalv']
data[cols] = data[cols].astype(int)

In [29]:
#find total senior veteran population and drop the gender separated data
data['seniorvet'] = data['mv65_74']+data['mv75+']+data['fv65_74']+data['fv75+']

In [30]:
data = data.drop(columns = ['mv65_74','mv75+','fv65_74','fv75+'])

In [31]:
#find total senior population in this series and drop the gender separated data
data['senior'] = data['m65_74']+data['m75+']+data['f65_74']+data['f75+']

In [32]:
data = data.drop(columns = ['m65_74','m75+','f65_74','f75+'])

In [33]:
#calculate percentages for each county
data['percentseniors_veteran'] = round((data['seniorvet']/data['senior'])*100, 2)
data['percentveterans_senior'] = round((data['seniorvet']/data['totalv'])*100, 2)

In [34]:
region = data[['seniorvet','senior','totalv','Region']].groupby('Region').sum()

In [35]:
region

,seniorvet,senior,totalv
Region,,,
GNRC,45370,263074,119735


In [36]:
#calculate percentages for region
region['percentseniors_veteran'] = round((region['seniorvet']/region['senior'])*100, 2)
region['percentveterans_senior'] = round((region['seniorvet']/region['totalv'])*100, 2)

In [37]:
region = region.reset_index()

In [38]:
#add columns to region to make it match data so we can concatenate and export - in order to import to MySidewalk
region['County'] = 'All'
region['GEOID'] = 'x'
region['StateFIPS'] = '47'
region['CountyFIPS'] = 'x'

In [39]:
frames = [data, region]
df = pd.concat(frames).reset_index(drop = True)

In [40]:
df.head(14)

,County,GEOID,totalv,StateFIPS,CountyFIPS,Region,seniorvet,senior,percentseniors_veteran,percentveterans_senior
0,"Stewart County, Tennessee",0500000US47161,1509,47,161,GNRC,568,2688,21.13,37.64
1,"Montgomery County, Tennessee",0500000US47125,28547,47,125,GNRC,5696,20125,28.30,19.95
2,"Houston County, Tennessee",0500000US47083,813,47,083,GNRC,370,1670,22.16,45.51
3,"Humphreys County, Tennessee",0500000US47085,1342,47,085,GNRC,630,3778,16.68,46.94
4,"Dickson County, Tennessee",0500000US47043,3141,47,043,GNRC,1298,8654,15.00,41.32
5,"Cheatham County, Tennessee",0500000US47021,2540,47,021,GNRC,1041,6137,16.96,40.98
6,"Robertson County, Tennessee",0500000US47147,3796,47,147,GNRC,1616,10616,15.22,42.57
7,"Sumner County, Tennessee",0500000US47165,11924,47,165,GNRC,5567,30562,18.22,46.69
8,"Davidson County, Tennessee",0500000US47037,28845,47,037,GNRC,13267,87560,15.15,45.99
9,"Wilson County, Tennessee",0500000US47189,8937,47,189,GNRC,4050,22300,18.16,45.32


In [41]:
county = df.loc[df['County'] == 'Humphreys County, Tennessee']
county

,County,GEOID,totalv,StateFIPS,CountyFIPS,Region,seniorvet,senior,percentseniors_veteran,percentveterans_senior
3,"Humphreys County, Tennessee",0500000US47085,1342,47,085,GNRC,630,3778,16.68,46.94


In [42]:
df.to_csv('../data/seniorveteranstatus.csv', index = False)